In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()
    #입력 이미지 채널 1개, 출력 채널 6개, 3X3의 정사각 컨볼루션 행렬
    #컨볼루션 커널 정의
    self.conv1 = nn.Conv2d(1,6,3)
    self.conv2 = nn.Conv2d(6,16,3)
    #아핀 (affine) 연산 : y = Wx + b
    self.fc1 = nn.Linear(16*6*6, 120) #6*6은 이미지 차원
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

  
  def forward(self,x):
    #(2,2) 크기의 윈도우에 대해 맥스 풀링(max pooling)
    x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
    #크기가 제곱수라면 하나의 숫자만을 특정
    x = F.max_pool2d(F.relu(self.conv2(x)),2)
    x= x.view(-1,self.num_flat_feature(x))
    x= F.relu(self.fc1(x))
    x= F.relu(self.fc2(x))
    x= self.fc3(x)
    return x
  
  def num_flat_feature(self,x):
    size = x.size()[1:] # 배치 차원을 제외한 모든 차원
    num_features = 1
    for s in size:
      num_features *= s
    return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


모델의 학습 가능한 매개변수들은 net.parameters()에 의해 반환

In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [3]:
print(params[1].size())

torch.Size([6])


In [4]:
input  = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[ 0.0619, -0.1187,  0.0085, -0.0201,  0.0772, -0.0558,  0.0628, -0.0540,
          0.0967, -0.1138]], grad_fn=<AddmmBackward>)


In [5]:
net.zero_grad()
out.backward(torch.randn(1,10))

손실 함수(Loss Function)

In [6]:
output = net(input)
target = torch.randn(10)
target = target.view(1,-1)
criterion = nn.MSELoss() #평균제곱오차

loss = criterion(output,target)

print(loss)

tensor(0.4665, grad_fn=<MseLossBackward>)


In [7]:
target = torch.randn(10)
print(target)
target = target.view(1,-1)
print(target)

tensor([-0.9231, -0.3744, -0.8557,  0.1418,  0.1753, -1.1366,  0.5112, -0.6396,
         0.1711,  1.5355])
tensor([[-0.9231, -0.3744, -0.8557,  0.1418,  0.1753, -1.1366,  0.5112, -0.6396,
          0.1711,  1.5355]])


역전파(Backpropagation)

변화도 누적을 피하기 위해 변화도 버퍼를 수동으로 0으로 설정이 필요

In [8]:
net.zero_grad() #모든 매개변수의 변화도를 0으로 초기화

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0012,  0.0048,  0.0057, -0.0092,  0.0051, -0.0047])


가중치 갱신

확률적 경사 하강법(SGD;Stochaastic Grdient Descent) : 새로운 가중치 = 가중치 - 학습률 * 변화도

In [16]:
learning_rate = 0.01
for f in net.parameters():
  f.data.sub_(f.grad.data*learning_rate)


import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr = 0.01)

optimizer.zero_grad()
ouput = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()

RuntimeError: ignored

In [13]:
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])